In [1]:
# Install and import necessary libraries
%pip install llama-cpp-python transformers sentencepiece protobuf pydantic hypertion

from hypertion import HyperFunction
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

### Create a `HyperFunction` instance

In [2]:
hyperfunction = HyperFunction()

### Define function and register it using `.takeover` method

In [9]:
import pathlib
from enum import Enum
from typing import Literal, NamedTuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Field


class ServerType(str, Enum):
    TESTING = "testing"
    DEVELOPMENT = "development"
    PRODUCTION = "production"

class ServerConfig(BaseModel):
    host: str = Field(default="http://localhost", description="The host url of the server.")
    port: int = Field(..., description="The port of the server.")
    use_ssl: bool = False

class LoggingConfig(TypedDict):
    """
    Logging Configuration
    :param level: Level of the logging.
    :param format: Logging format.
    """
    level: Literal['debug', 'info', 'warning', 'error', 'critical'] = 'info'
    format: str

class User(NamedTuple):
    """
    User Info
    :param name: Name of the user.
    :param role: Role of the user.
    """
    name: str
    role: Literal['admin', 'developer', 'tester'] = 'developer'

@hyperfunction.takeover
def setup_server(
    name: str,
    server_type: ServerType,
    server_config: ServerConfig,
    log_config: LoggingConfig,
    users: list[User],
    persist_at: pathlib.Path,
) -> None:
    """
    Setup server with the given configuration.

    @param name: The name of the server.
    @param server_type: The type of the server.
    @param server_config: Configuration for the server.
    @param log_config: Logging configuration.
    @param users: List of users with their respective roles.
    @param persist_at: Path to persist server log.
    """
    print(f"Setting up server '{name}' of type '{server_type}'...")
    print(f"Server Configuration: {server_config}")
    print(f"Logging Configuration: {log_config}")
    print(f"Users: {users}")
    print(f"Persist location: {persist_at}")

### Display registered functions

In [10]:
hyperfunction.registry()

setup_server(
   name: str,
   server_type: ServerType,
   server_config: ServerConfig,
   log_config: LoggingConfig,
   users: list[User],
   persist_at: Path
) -> None:
"""Setup server with the given configuration."""


### Build function schema with specified LLM format

In [11]:
print(hyperfunction.format('functionary', as_json=True))

[
    {
        "type": "function",
        "function": {
            "name": "setup_server",
            "description": "Setup server with the given configuration.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the server."
                    },
                    "server_type": {
                        "type": "string",
                        "enum": [
                            "TESTING",
                            "DEVELOPMENT",
                            "PRODUCTION"
                        ],
                        "description": "The type of the server."
                    },
                    "server_config": {
                        "type": "object",
                        "properties": {
                            "host": {
                                "type": "string",
                   

### Setup LLM

In [4]:
llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.4-GGUF",
    filename="functionary-small-v2.4.Q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.4-GGUF"),
    n_gpu_layers=-1,
    n_ctx=4096,
    verbose=False
)

def get_functionary_metadata(query: str, functions: list[dict]):
    result = llm.create_chat_completion(
        messages = [{'role': 'user', 'content': query}],
        tools=functions,
        tool_choice="auto",
    )

    f = result["choices"][0]["message"]['tool_calls'][0]['function']
    return f['name'], f['arguments']

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Invoke `Functionary` and get the function metadata

In [17]:
QUERY = """\
Setup a server of type development on port number 8001 with ssl enabled.
Logging format should be "%(asctime)s - %(name)s - %(levelname)s - %(message)s".
Add user synacktra with developer role and 1littlecoder with tester role.
The server logs should be persisted at /var/logs.\
"""
name, arguments = get_functionary_metadata(query=QUERY, functions=hyperfunction.format('functionary'))
print(f"Name: {name}\nRaw arguments: {arguments}")

c:\Users\synac\AppData\Local\Programs\Python\Python310\lib\site-packages\llama_cpp\llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Name: setup_server
Raw arguments: {"name": "development-server", "server_type": "DEVELOPMENT", "server_config": {"port": 8001, "use_ssl": true}, "log_config": {"format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"}, "users": [{"name": "synacktra", "role": "developer"}, {"name": "1littlecoder", "role": "tester"}], "persist_at": "/var/logs"}


### Compose the metadata as Function-Call object

In [18]:
function_call = hyperfunction.compose(name=name, arguments=arguments)
print(function_call)

setup_server(
   name='development-server',
   server_type=<ServerType.DEVELOPMENT: 'development'>,
   server_config=ServerConfig(host='http://localhost', port=8001, use_ssl=True),
   log_config={'level': 'info', 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'},
   users=[('synacktra', 'developer'), ('1littlecoder', 'tester')],
   persist_at=WindowsPath('/var/logs')
)


### Invoke the function

In [19]:
function_call()

Setting up server 'development-server' of type 'development'...
Server Configuration: host='http://localhost' port=8001 use_ssl=True
Logging Configuration: {'level': 'info', 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}
Users: [('synacktra', 'developer'), ('1littlecoder', 'tester')]
Persist location: \var\logs
